# Split Data into Train - Val - Eval

Train set -> England 90%

Val set -> England 10% (internal validation)

Eval set -> Wales & Scotland (External validation)

In [1]:
#uncomment this below code to install imblearn package
# !pip install imbalanced-learn

In [2]:
# !pip install scikit-optimize

In [3]:
import pandas as pd
import numpy as np
import sklearn

#statistics
from scipy.stats import chi2_contingency, ttest_ind

import cudf #gpu-powered DataFrame (Pandas alternative)

#imbalance handling
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler, RepeatedEditedNearestNeighbours
from imblearn.pipeline import Pipeline

#preprocessing
from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

#hyperparameter search
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from skopt import gp_minimize

#internal validation
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedStratifiedKFold, cross_val_score, GridSearchCV, PredefinedSplit, RandomizedSearchCV


#performance metrices
from sklearn.metrics import make_scorer, confusion_matrix, classification_report, f1_score, balanced_accuracy_score, r2_score, auc, average_precision_score, roc_auc_score, recall_score, roc_curve, accuracy_score

#Models selection
from sklearn.naive_bayes import GaussianNB, ComplementNB
from sklearn.linear_model import LogisticRegression, ElasticNet
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from cuml.svm import SVC #gpu-powered SVM


#save and load trained model
import pickle

#visualisation
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter

import os

In [4]:
# Data loader
features = pd.read_csv("../FinalData/cleaned_features_03102023.csv")
outcomes = pd.read_csv("../FinalData/cleaned_outcomes_11072023.csv")
features = features[features.columns[1:]]
# outcomes = outcomes[outcomes.columns[1:]]

/tmp/ipykernel_18219/1470992490.py:2: DtypeWarning: Columns (26,27,29,30,33,34,80,84) have mixed types. Specify dtype option on import or set low_memory=False.
  features = pd.read_csv("../FinalData/cleaned_features_03102023.csv")


In [5]:
print(features.shape)
print(outcomes.shape)

(580607, 99)
(577962, 15)


In [6]:
features.columns

Index(['patid', 'practice_id', 'sex', 'age', 'BMI', 'weight', 'height',
       'ethnicity', 'ethnic_group', 'smokingStatus', 'CharlsonScore',
       'count_rhinitis', 'count_cardiovascular', 'count_heartfailure',
       'count_psoriasis', 'count_anaphylaxis', 'count_diabetes', 'count_ihd',
       'count_anxiety', 'count_eczema', 'count_nasalpolyps',
       'count_paracetamol', 'count_nsaids', 'count_betablocker', 'id',
       'event_date', 'recorded_date', 'visit_id', 'code_id', 'snomed_id',
       'numeric_1', 'numeric_2', 'created_datetime', 'updated_datetime',
       'PEFStatus', 'EosinophilLevel', 'BTS_step', 'average_daily_dose_ICS',
       'prescribed_daily_dose_ICS', 'ICS_medication_possesion_ratio',
       'DeviceType', 'Spacer', 'numOCS', 'PriorEducation', 'numPCS',
       'numPCSAsthma', 'numAntibioticsEvents', 'numAntibioticswithLRTI',
       'numOCSEvents', 'numOCSwithLRTI', 'numAsthmaAttacks',
       'numAcuteRespEvents', 'numHospEvents', 'month_12', 'month_4', 'month_5',


In [7]:
features.head()

,patid,practice_id,sex,age,BMI,weight,height,ethnicity,ethnic_group,smokingStatus,...,psoriasis,anaphylaxis,diabetes,ihd,anxiety,eczema,nasalpolyps,paracetamol,nsaids,betablocker
0,43231452,39,0.0,48,26.609713,76.0,1.6900,not_recorded,not_recorded,Active Smoker,...,0,0,0,0,0,0,0,0,0,0
1,43206365,39,1.0,58,23.946360,72.5,1.7400,not_recorded,not_recorded,Former Smoker,...,0,0,0,0,1,0,0,0,0,0
2,43203606,559,0.0,51,17.104513,39.0,1.5100,not_recorded,not_recorded,Active Smoker,...,0,0,0,0,0,1,0,0,0,0
3,43117348,502,0.0,69,35.303241,74.0,1.4478,not_recorded,not_recorded,Former Smoker,...,1,0,0,0,0,1,0,0,0,0
4,43289035,593,0.0,49,31.217482,75.0,1.5500,White British,White - ethnic group,Active Smoker,...,0,0,0,0,0,1,0,0,0,0


In [8]:
outcomes.head()

,patid,outcome_3months,outcome_6months,outcome_9months,outcome_12months,outcome_15months,outcome_18months,outcome_21months,outcome_24months,outcome_combined_6months,outcome_combined_9months,outcome_combined_12months,outcome_combined_15months,outcome_combined_18months,outcome_combined_24months
0,43231452,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,43206365,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,43203606,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,43117348,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,43289035,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
masterData = features.merge(outcomes, how = 'left', left_on='patid', right_on='patid') #join table
# masterData = masterData.dropna() #NAs from Country
masterData = masterData.reset_index(drop=True)
exclude_columns = ['weight', 'height', 'id', 'event_date', 'recorded_date', 'visit_id', 'code_id', 'snomed_id',
       'numeric_1', 'numeric_2', 'created_datetime', 'updated_datetime',]
masterData = masterData.loc[:,~masterData.columns.isin(exclude_columns)]
print('original data shape: ', masterData.shape)

original data shape:  (580607, 101)


In [10]:
pd.options.display.max_rows = 287
masterData.isna().sum()

patid                                 0
practice_id                           0
sex                                   0
age                                   0
BMI                                   0
ethnicity                             0
ethnic_group                          0
smokingStatus                         0
CharlsonScore                         0
count_rhinitis                        0
count_cardiovascular                  0
count_heartfailure                    0
count_psoriasis                       0
count_anaphylaxis                     0
count_diabetes                        0
count_ihd                             0
count_anxiety                         0
count_eczema                          0
count_nasalpolyps                     0
count_paracetamol                     0
count_nsaids                          0
count_betablocker                     0
PEFStatus                             0
EosinophilLevel                       0
BTS_step                              0


In [11]:
#Positive vs negative class ratio

print('3 months -> 1 : ', round(masterData.outcome_3months.value_counts()[0]/masterData.outcome_3months.value_counts()[1],2))
print('6 months -> 1 : ', round(masterData.outcome_combined_6months.value_counts()[0]/masterData.outcome_combined_6months.value_counts()[1],2))
print('9 months -> 1 : ', round(masterData.outcome_combined_9months.value_counts()[0]/masterData.outcome_combined_9months.value_counts()[1],2))
print('12 months -> 1 : ', round(masterData.outcome_combined_12months.value_counts()[0]/masterData.outcome_combined_12months.value_counts()[1],2))
print('15 months -> 1 : ', round(masterData.outcome_combined_15months.value_counts()[0]/masterData.outcome_combined_15months.value_counts()[1],2))
print('18 months -> 1 : ', round(masterData.outcome_combined_18months.value_counts()[0]/masterData.outcome_combined_18months.value_counts()[1],2))
print('24 months -> 1 : ', round(masterData.outcome_combined_24months.value_counts()[0]/masterData.outcome_combined_24months.value_counts()[1],2))


3 months -> 1 :  34.02
6 months -> 1 :  22.54
9 months -> 1 :  17.42
12 months -> 1 :  13.04
15 months -> 1 :  10.6
18 months -> 1 :  9.64
24 months -> 1 :  8.61


In [12]:
#Proportion of asthma attack in each outcome

print('3 months -> ', round(masterData.outcome_3months.value_counts()[1]/len(masterData)*100,2), '%')
print('6 months -> ', round(masterData.outcome_combined_6months.value_counts()[1]/len(masterData)*100,2), '%')
print('9 months -> ', round(masterData.outcome_combined_9months.value_counts()[1]/len(masterData)*100,2), '%')
print('12 months -> ', round(masterData.outcome_combined_12months.value_counts()[1]/len(masterData)*100,2), '%')
print('15 months -> ', round(masterData.outcome_combined_15months.value_counts()[1]/len(masterData)*100,2), '%')
print('18 months -> ', round(masterData.outcome_combined_18months.value_counts()[1]/len(masterData)*100,2), '%')
print('24 months -> ', round(masterData.outcome_combined_24months.value_counts()[1]/len(masterData)*100,2), '%')

3 months ->  2.86 %
6 months ->  4.25 %
9 months ->  5.43 %
12 months ->  7.12 %
15 months ->  8.62 %
18 months ->  9.4 %
24 months ->  10.41 %


In [13]:
#Data scenario
# 1: all data without ethnicity variable
# 2: all data with ethnicity variable (include all missing values in ethnicity as separate group)
# 3: filter data based on ethnicity (exclude missing values)

scenario = 1 #change it based on the scenario

if scenario == 1:
    #include all data
    allData = masterData
    
elif scenario == 2:
    #Exclude ethnic column
    allData = masterData.drop('ethnic_group', axis=1)
    
elif scenario == 3:
    #exclude missing values for ethnic variable
    allData = masterData[masterData.ethnic_group!='not_recorded']
    
allData = allData.reset_index(drop=True)
print('Data shape for scenario', str(scenario), allData.shape)



Data shape for scenario 1 (580607, 101)


In [14]:
#Split data into training, validation, and evaluation set based on the country. Include only 18+ patients.

trainingData = allData[(allData.Country == 'England') & (allData.age>=18)]
trainingData, validationData = train_test_split(trainingData, test_size=0.1, stratify=trainingData['outcome_combined_12months'])

In [15]:
#Proportion of asthma attack in each outcome

print('3 months -> ', round(trainingData.outcome_3months.value_counts()[1]/len(trainingData)*100,2), '%')
print('6 months -> ', round(trainingData.outcome_combined_6months.value_counts()[1]/len(trainingData)*100,2), '%')
print('9 months -> ', round(trainingData.outcome_combined_9months.value_counts()[1]/len(trainingData)*100,2), '%')
print('12 months -> ', round(trainingData.outcome_combined_12months.value_counts()[1]/len(trainingData)*100,2), '%')
print('15 months -> ', round(trainingData.outcome_combined_15months.value_counts()[1]/len(trainingData)*100,2), '%')
print('18 months -> ', round(trainingData.outcome_combined_18months.value_counts()[1]/len(trainingData)*100,2), '%')
print('24 months -> ', round(trainingData.outcome_combined_24months.value_counts()[1]/len(trainingData)*100,2), '%')

3 months ->  2.8 %
6 months ->  4.17 %
9 months ->  5.34 %
12 months ->  7.02 %
15 months ->  8.51 %
18 months ->  9.29 %
24 months ->  10.29 %


In [16]:
#Proportion of asthma attack in each outcome

print('3 months -> ', round(validationData.outcome_3months.value_counts()[1]/len(validationData)*100,2), '%')
print('6 months -> ', round(validationData.outcome_combined_6months.value_counts()[1]/len(validationData)*100,2), '%')
print('9 months -> ', round(validationData.outcome_combined_9months.value_counts()[1]/len(validationData)*100,2), '%')
print('12 months -> ', round(validationData.outcome_combined_12months.value_counts()[1]/len(validationData)*100,2), '%')
print('15 months -> ', round(validationData.outcome_combined_15months.value_counts()[1]/len(validationData)*100,2), '%')
print('18 months -> ', round(validationData.outcome_combined_18months.value_counts()[1]/len(validationData)*100,2), '%')
print('24 months -> ', round(validationData.outcome_combined_24months.value_counts()[1]/len(validationData)*100,2), '%')

3 months ->  2.85 %
6 months ->  4.2 %
9 months ->  5.37 %
12 months ->  7.02 %
15 months ->  8.54 %
18 months ->  9.33 %
24 months ->  10.32 %


In [17]:
#Split data into training and evaluation set based on the country. Include only 18+ patients.

evaluationData = allData[((allData.Country == 'Scotland') | (allData.Country == 'Wales')) & (allData.age>=18)] #used for validation
evaluationDataWales = allData[(allData.Country == 'Wales') & (allData.age>=18)] #used for validation
evaluationDataScotland = allData[(allData.Country == 'Scotland') & (allData.age>=18)] #used for validation

#remove country variable
trainingData = trainingData.drop('Country', axis=1)
validationData = validationData.drop('Country', axis=1)
evaluationData = evaluationData.drop('Country', axis=1)
evaluationDataWales = evaluationDataWales.drop('Country', axis=1)
evaluationDataScotland = evaluationDataScotland.drop('Country', axis=1)

trainingData = trainingData.reset_index(drop=True)
validationData = validationData.reset_index(drop=True)
evaluationData = evaluationData.reset_index(drop=True)
evaluationDataWales = evaluationDataWales.reset_index(drop=True)
evaluationDataScotland = evaluationDataScotland.reset_index(drop=True)

print('Training data shape:', trainingData.shape)
print('Validation data shape:', validationData.shape)
print('Evaluation data shape: ', evaluationData.shape)
print('Evaluation data Wales shape: ', evaluationDataWales.shape)
print('Evaluation data Scotland shape: ', evaluationDataScotland.shape)

Training data shape: (502003, 100)
Validation data shape: (55779, 100)
Evaluation data shape:  (22825, 100)
Evaluation data Wales shape:  (14485, 100)
Evaluation data Scotland shape:  (8340, 100)


In [18]:
#Save original data
trainingData.to_csv('../FinalData/trainingData_08102023.csv', index_label=False, index=False)
validationData.to_csv('../FinalData/validationData_08102023.csv', index_label=False, index=False)
evaluationData.to_csv('../FinalData/evaluationData_08102023.csv', index_label=False, index=False)
evaluationDataWales.to_csv('../FinalData/evaluationDataWales_08102023.csv', index_label=False, index=False)
evaluationDataScotland.to_csv('../FinalData/evaluationDataScotland_08102023.csv', index_label=False, index=False)

In [19]:
#encode categorical data

cat_vars = ['PEFStatus','EosinophilLevel']
onehot_vars = ['ethnic_group','smokingStatus', 'DeviceType', 'cat_BMI', 'imd_decile']
data_categorical = trainingData[cat_vars]
data_onehot = trainingData[onehot_vars]

#ordinal encoder
encoder = OrdinalEncoder(categories=[['not_recorded','less than 60', '60-80', 'more than 80'], ['unknown', 'normal', 'high']]).set_output(transform="pandas")
data_encoded = encoder.fit_transform(data_categorical)
pickle.dump(encoder, open('../Models/cat_encoder.pkl', 'wb'))
    
#one hot encoder
onehot_encoder = OneHotEncoder(sparse_output=False).set_output(transform="pandas")
onehot_encoded = onehot_encoder.fit_transform(data_onehot)
pickle.dump(onehot_encoder, open('../Models/onehot_encoder.pkl', 'wb'))

trainingData = pd.concat([trainingData.drop(cat_vars, axis=1), data_encoded], axis=1)
trainingData = pd.concat([trainingData.drop(onehot_vars, axis=1), onehot_encoded], axis=1)

print('Data shape after encoding: ', trainingData.shape)

Data shape after encoding:  (502003, 125)


In [20]:
#encode cat vars for validation and evaluation set
data_val_categorical = validationData[cat_vars]
data_val_onehot = validationData[onehot_vars]
data_eval_categorical = evaluationData[cat_vars]
data_eval_onehot = evaluationData[onehot_vars]
data_eval_Wales_categorical = evaluationDataWales[cat_vars]
data_eval_Wales_onehot = evaluationDataWales[onehot_vars]
data_eval_Scotland_categorical = evaluationDataScotland[cat_vars]
data_eval_Scotland_onehot = evaluationDataScotland[onehot_vars]

encoder = pickle.load(open('../Models/cat_encoder.pkl', 'rb'))
data_val_encoded = encoder.transform(data_val_categorical)
data_eval_encoded = encoder.transform(data_eval_categorical)
data_eval_Wales_encoded = encoder.transform(data_eval_Wales_categorical)
data_eval_Scotland_encoded = encoder.transform(data_eval_Scotland_categorical)

onehot_encoder = pickle.load(open('../Models/onehot_encoder.pkl', 'rb'))
onehot_val_encoded = onehot_encoder.transform(data_val_onehot)
onehot_eval_encoded = onehot_encoder.transform(data_eval_onehot)
onehot_eval_Wales_encoded = onehot_encoder.transform(data_eval_Wales_onehot)
onehot_eval_Scotland_encoded = onehot_encoder.transform(data_eval_Scotland_onehot)

validationData = pd.concat([validationData.drop(cat_vars, axis=1), data_val_encoded], axis=1)
validationData = pd.concat([validationData.drop(onehot_vars, axis=1), onehot_val_encoded], axis=1)

evaluationData = pd.concat([evaluationData.drop(cat_vars, axis=1), data_eval_encoded], axis=1)
evaluationData = pd.concat([evaluationData.drop(onehot_vars, axis=1), onehot_eval_encoded], axis=1)

evaluationDataWales = pd.concat([evaluationDataWales.drop(cat_vars, axis=1), data_eval_Wales_encoded], axis=1)
evaluationDataWales = pd.concat([evaluationDataWales.drop(onehot_vars, axis=1), onehot_eval_Wales_encoded], axis=1)

evaluationDataScotland = pd.concat([evaluationDataScotland.drop(cat_vars, axis=1), data_eval_Scotland_encoded], axis=1)
evaluationDataScotland = pd.concat([evaluationDataScotland.drop(onehot_vars, axis=1), onehot_eval_Scotland_encoded], axis=1)

print('Val data shape after encoding: ', validationData.shape)
print('Eval data shape after encoding: ', evaluationData.shape)
print('Evaluation data Wales shape: ', evaluationDataWales.shape)
print('Evaluation data Scotland shape: ', evaluationDataScotland.shape)

Val data shape after encoding:  (55779, 125)
Eval data shape after encoding:  (22825, 125)
Evaluation data Wales shape:  (14485, 125)
Evaluation data Scotland shape:  (8340, 125)


In [21]:
#Data normalisation for continous variable into 0-1 range


continuous_vars = ['age', 'CharlsonScore', 'average_daily_dose_ICS', 'prescribed_daily_dose_ICS', 'ICS_medication_possesion_ratio', 
                   'numOCS', 'numPCS', 'numPCSAsthma', 'numAntibioticsEvents', 'numAntibioticswithLRTI', 'numOCSEvents', 'numOCSwithLRTI', 
                   'numAsthmaAttacks', 'numAcuteRespEvents', 'numHospEvents']

# define scaler
scaler = StandardScaler()
data_scaled = scaler.fit_transform(trainingData[continuous_vars])
pickle.dump(scaler, open('../Models/cont_scaler.pkl', 'wb'))


data_scaled = pd.DataFrame(data_scaled, columns=scaler.get_feature_names_out())
trainingData = pd.concat([trainingData.drop(continuous_vars, axis=1), data_scaled], axis=1)

print('Data shape after scaling: ', trainingData.shape)

Data shape after scaling:  (502003, 125)


In [22]:
#Normalisation for evaluation data

scaler = pickle.load(open('../Models/cont_scaler.pkl', 'rb'))
data_val_scaled = scaler.transform(validationData[continuous_vars])
data_eval_scaled = scaler.transform(evaluationData[continuous_vars])
data_eval_Wales_scaled = scaler.transform(evaluationDataWales[continuous_vars])
data_eval_Scotland_scaled = scaler.transform(evaluationDataScotland[continuous_vars])

data_val_scaled = pd.DataFrame(data_val_scaled, columns=scaler.get_feature_names_out())
validationData = pd.concat([validationData.drop(continuous_vars, axis=1), data_val_scaled], axis=1)

data_eval_scaled = pd.DataFrame(data_eval_scaled, columns=scaler.get_feature_names_out())
evaluationData = pd.concat([evaluationData.drop(continuous_vars, axis=1), data_eval_scaled], axis=1)

data_eval_Wales_scaled = pd.DataFrame(data_eval_Wales_scaled, columns=scaler.get_feature_names_out())
evaluationDataWales = pd.concat([evaluationDataWales.drop(continuous_vars, axis=1), data_eval_Wales_scaled], axis=1)

data_eval_Scotland_scaled = pd.DataFrame(data_eval_Scotland_scaled, columns=scaler.get_feature_names_out())
evaluationDataScotland = pd.concat([evaluationDataScotland.drop(continuous_vars, axis=1), data_eval_Scotland_scaled], axis=1)

print('Val data shape after scaling: ', validationData.shape)
print('Eval data shape after scaling: ', evaluationData.shape)
print('Evaluation data Wales shape: ', evaluationDataWales.shape)
print('Evaluation data Scotland shape: ', evaluationDataScotland.shape)

Val data shape after scaling:  (55779, 125)
Eval data shape after scaling:  (22825, 125)
Evaluation data Wales shape:  (14485, 125)
Evaluation data Scotland shape:  (8340, 125)


In [23]:
#Define feature candidates

features_columns = trainingData.columns.to_list()
exclude_columns = ['patid', 'practice_id', #identifier
                   'BMI', #use the categorical instead
                   'ethnicity', #use ethnic_group instead
                   'Spacer',  #all zero
                   
                   'outcome_3months', 'outcome_6months', 'outcome_9months', 'outcome_12months', 'outcome_15months', 'outcome_18months', 
                   'outcome_21months', 'outcome_24months', 'outcome_combined_6months', 'outcome_combined_9months', 'outcome_combined_12months', 
                   'outcome_combined_15months', 'outcome_combined_18months', 'outcome_combined_24months', '3months', '6months', '12months', '24months', #outcomes variable
                   
                   'postcode_district', 'County', 'LocalAuthority', 'OutputAreaClassification', #location related variables, use IMD decile only
                   
                   'cat_age', 'cat_average_daily_dose_ICS', 'cat_prescribed_daily_dose_ICS', 'cat_ICS_medication_possesion_ratio', 'cat_numOCS', 'cat_numOCSEvents', 
                   'cat_numOCSwithLRTI', 'cat_numAcuteRespEvents', 'cat_numAntibioticsEvents', 'cat_numAntibioticswithLRTI', 'cat_numAsthmaAttacks', 'cat_numHospEvents', 
                   'cat_numPCS', 'cat_numPCSAsthma', #use continous vars instead
                   
                   'count_rhinitis', 'count_cardiovascular', 'count_heartfailure',
                   'count_psoriasis', 'count_anaphylaxis', 'count_diabetes', 'count_ihd',
                   'count_anxiety', 'count_eczema', 'count_nasalpolyps',
                   'count_paracetamol', 'count_nsaids', 'count_betablocker', #use binary ones
                   
                   'paracetamol', 'nsaids', 'betablocker', #no data in evaluation
                   
                   'numOCSEvents', #duplicate with numOCS
                  ]
exclude_columns = exclude_columns + [x for x in features_columns if '_count' in x] #filter out commorbid count variables
features_columns = [x for x in features_columns if x not in exclude_columns]
print('Features size: ', len(features_columns))
print(features_columns)

Features size:  71
['sex', 'BTS_step', 'PriorEducation', 'month_12', 'month_4', 'month_5', 'month_10', 'month_1', 'month_6', 'month_3', 'month_11', 'month_8', 'month_9', 'month_7', 'month_2', 'rhinitis', 'cardiovascular', 'heartfailure', 'psoriasis', 'anaphylaxis', 'diabetes', 'ihd', 'anxiety', 'eczema', 'nasalpolyps', 'PEFStatus', 'EosinophilLevel', 'ethnic_group_Asian - ethnic group', 'ethnic_group_Black - ethnic group', 'ethnic_group_Mixed ethnic census group', 'ethnic_group_Other ethnic group', 'ethnic_group_White - ethnic group', 'ethnic_group_not_recorded', 'smokingStatus_Active Smoker', 'smokingStatus_Former Smoker', 'smokingStatus_Non Smoker', 'DeviceType_BAI', 'DeviceType_DPI', 'DeviceType_NEB', 'DeviceType_pMDI', 'DeviceType_unknown', 'cat_BMI_normal', 'cat_BMI_not recorded', 'cat_BMI_obese', 'cat_BMI_overweight', 'cat_BMI_underweight', 'imd_decile_0', 'imd_decile_1', 'imd_decile_2', 'imd_decile_3', 'imd_decile_4', 'imd_decile_5', 'imd_decile_6', 'imd_decile_7', 'imd_decile_8

In [24]:
#Save encoded data
trainingData.to_csv('../FinalData/trainingDataEncoded_08102023.csv', index_label=False, index=False)
validationData.to_csv('../FinalData/validationDataEncoded_08102023.csv', index_label=False, index=False)
evaluationData.to_csv('../FinalData/evaluationDataEncoded_08102023.csv', index_label=False, index=False)
evaluationDataWales.to_csv('../FinalData/evaluationDataWalesEncoded_08102023.csv', index_label=False, index=False)
evaluationDataScotland.to_csv('../FinalData/evaluationDataScotlandEncoded_08102023.csv', index_label=False, index=False)